In [1]:
import pandas as pd
import os
pd.options.display.max_colwidth = 100
os.chdir('../')
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

from notebooks.utils.load_data import load_psdn_ocean_transfers, load_lp_staking, load_psdn_ocean_staking, load_psdn_transfers
psdn_ocean_contract_address = '0x51Fa2efd62ee56a493f24AE963eAce7D0051929E'
lp_staking_address = '0x08D3f2cAe4eaCDCbFb0f7FfaD07a6d00FC36Db6E' # psdnOcean-ocean lp
psdn_ocean_staking_contract_address = '0xEb1CEFa6F175B4c889582DFEea4ee155CCD5D2A5'

# load datasets
psdn_ocean_transfers = load_psdn_ocean_transfers()
psdn_ocean_staking = load_psdn_ocean_staking()
lp_staking = load_lp_staking()
psdn_transfers = load_psdn_transfers()

## psdnOCEAN minting and where did they go?

In [16]:
total_psdn_ocean_minted = psdn_ocean_transfers.loc[(psdn_ocean_transfers["From"] == "0x0000000000000000000000000000000000000000")]['Quantity'].sum()
print(f"Total psdnOcean minted: {total_psdn_ocean_minted.round(0)}")

Total psdnOcean minted: 2057187.0


In [5]:
df = psdn_ocean_transfers.loc[(psdn_ocean_transfers["From"] == "0x0000000000000000000000000000000000000000") & (psdn_ocean_transfers["Method"] == "Deposit")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='psdnOcean minted'))

df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Join Pool") & (psdn_ocean_transfers['To'] == "0xba12222222228d8ba445958a75a0704d566bf2c8")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()#['To'].value_counts()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='psdnOcean join LP'))

df = psdn_ocean_transfers.loc[ (psdn_ocean_transfers["Method"] == "Stake")].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()
fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='psdnOcean single stake'))

# Edit the layout
fig.update_layout(title='psdnOcean flow weekly',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()


## Ocean rewards to stakers

In [19]:
total_lp_staking_ocean_rewards = lp_staking.loc[(lp_staking['TokenSymbol'] == 'OCEAN') & (lp_staking['From'] == lp_staking_address.lower())]['TokenValue'].sum()
total_psdn_ocean_staking_ocean_rewards = psdn_ocean_staking.loc[(psdn_ocean_staking['TokenSymbol'] == 'OCEAN') & (psdn_ocean_staking['From'] ==psdn_ocean_staking_contract_address.lower())].sum()['TokenValue']

print(f"Total Ocean rewards for lp stakers: {total_lp_staking_ocean_rewards.round(0)}")
print(f"Total Ocean rewards for psdnOcean stakers: {total_psdn_ocean_staking_ocean_rewards.round(0)}")

Total Ocean rewards for lp stakers: 65753.0
Total Ocean rewards for psdnOcean stakers: 19630.0


In [18]:
# Create traces
fig = go.Figure()

df = lp_staking.loc[(lp_staking['TokenSymbol'] == 'OCEAN') & (lp_staking['From'] ==lp_staking_address.lower())].groupby(pd.Grouper(key='DateTime', freq='W'))['TokenValue'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['TokenValue'],
                    mode='lines',
                    name='To LP stakers'))

df = psdn_ocean_staking.loc[(psdn_ocean_staking['TokenSymbol'] == 'OCEAN') & (psdn_ocean_staking['From'] ==psdn_ocean_staking_contract_address.lower())].groupby(pd.Grouper(key='DateTime', freq='W'))['TokenValue'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['TokenValue'],
                    mode='lines',
                    name='To psdnOcean stakers'))

# Edit the layout
fig.update_layout(title='Ocean rewards to stakers weekly',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()


## Psdn rewards to stakers

In [20]:
total_lp_staking_psdn_rewards = psdn_transfers.loc[psdn_transfers['From'] == '0x2337e518542db50da49f686fe3db883908c3a517']['Quantity'].sum()
total_psdn_ocean_staking_psdn_rewards = psdn_transfers.loc[psdn_transfers['From'] == '0x5c87da6a4947b49842c33a82bc30bac3a6811e64']['Quantity'].sum()

print(f"Total Psdn rewards for lp stakers: {total_lp_staking_psdn_rewards.round(0)}")
print(f"Total Psdn rewards for psdnOcean stakers: {total_psdn_ocean_staking_psdn_rewards.round(0)}")

Total Psdn rewards for lp stakers: 3470.0
Total Psdn rewards for psdnOcean stakers: 5243.0


In [21]:
fig = go.Figure()
df = psdn_transfers.loc[psdn_transfers['From'] == '0x2337e518542db50da49f686fe3db883908c3a517'].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='To LP stakers'))

df = psdn_transfers.loc[psdn_transfers['From'] == '0x5c87da6a4947b49842c33a82bc30bac3a6811e64'].groupby(pd.Grouper(key='DateTime', freq='W'))['Quantity'].sum().reset_index()

fig.add_trace(go.Scatter(x=df['DateTime'], y=df['Quantity'],
                    mode='lines',
                    name='To psdnOcean stakers'))

# Edit the layout
fig.update_layout(title='PSDN rewards to stakers weekly',
                   xaxis_title='Day',
                   yaxis_title='Amount', 
                   autosize=True,
                    width=1000,
                    height=600,
    )
fig.show()
